In [1]:
pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 29.4 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 42.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 31.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 27.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 1.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 28.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 58.2 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: n

In [16]:
import os
import glob
import random
from pathlib import Path

import cv2
import numpy as np
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

from ultralytics import YOLO
from ultralytics.models.yolo.detect.train import DetectionTrainer
from ultralytics.utils.loss import v8DetectionLoss

from types import SimpleNamespace

In [17]:
# Official Facebook ConvNext Implementation
class DropPath(nn.Module):
    def __init__(self, drop_prob=None):
        super(DropPath, self).__init__()
        self.drop_prob = drop_prob

    def forward(self, x):
        if self.drop_prob == 0. or not self.training:
            return x
        keep_prob = 1 - self.drop_prob
        shape = (x.shape[0],) + (1,) * (x.ndim - 1)
        random_tensor = keep_prob + torch.rand(shape, dtype=x.dtype, device=x.device)
        random_tensor.floor_()
        output = x.div(keep_prob) * random_tensor
        return output


In [18]:
class LayerNorm(nn.Module):
    def __init__(self, normalized_shape, eps=1e-6, data_format="channels_last"):
        super().__init__()
        self.weight = nn.Parameter(torch.ones(normalized_shape))
        self.bias = nn.Parameter(torch.zeros(normalized_shape))
        self.eps = eps
        self.data_format = data_format
        if self.data_format not in ["channels_last", "channels_first"]:
            raise NotImplementedError 
        self.normalized_shape = (normalized_shape, )
    
    def forward(self, x):
        if self.data_format == "channels_last":
            return F.layer_norm(x, self.normalized_shape, self.weight, self.bias, self.eps)
        elif self.data_format == "channels_first":
            u = x.mean(1, keepdim=True)
            s = (x - u).pow(2).mean(1, keepdim=True)
            x = (x - u) / torch.sqrt(s + self.eps)
            x = self.weight[:, None, None] * x + self.bias[:, None, None]
            return x


In [19]:
class Block(nn.Module):
    def __init__(self, dim, drop_path=0., layer_scale_init_value=1e-6):
        super().__init__()
        self.dwconv = nn.Conv2d(dim, dim, kernel_size=7, padding=3, groups=dim)
        self.norm = LayerNorm(dim, eps=1e-6)
        self.pwconv1 = nn.Linear(dim, 4 * dim)
        self.act = nn.GELU()
        self.pwconv2 = nn.Linear(4 * dim, dim)
        self.gamma = nn.Parameter(layer_scale_init_value * torch.ones((dim)), 
                                    requires_grad=True) if layer_scale_init_value > 0 else None
        self.drop_path = DropPath(drop_path) if drop_path > 0. else nn.Identity()

    def forward(self, x):
        input = x
        x = self.dwconv(x)
        x = x.permute(0, 2, 3, 1)  # (N, C, H, W) -> (N, H, W, C)
        x = self.norm(x)
        x = self.pwconv1(x)
        x = self.act(x)
        x = self.pwconv2(x)
        if self.gamma is not None:
            x = self.gamma * x
        x = x.permute(0, 3, 1, 2)  # (N, H, W, C) -> (N, C, H, W)
        x = input + self.drop_path(x)
        return x


In [20]:

class ConvNeXt(nn.Module):
    def __init__(self, in_chans=3, num_classes=1000, 
                 depths=[3, 3, 9, 3], dims=[96, 192, 384, 768], drop_path_rate=0., 
                 layer_scale_init_value=1e-6, head_init_scale=1.):
        super().__init__()

        self.downsample_layers = nn.ModuleList()
        stem = nn.Sequential(
            nn.Conv2d(in_chans, dims[0], kernel_size=4, stride=4),
            LayerNorm(dims[0], eps=1e-6, data_format="channels_first")
        )
        self.downsample_layers.append(stem)
        
        for i in range(3):
            downsample_layer = nn.Sequential(
                    LayerNorm(dims[i], eps=1e-6, data_format="channels_first"),
                    nn.Conv2d(dims[i], dims[i+1], kernel_size=2, stride=2),
            )
            self.downsample_layers.append(downsample_layer)

        self.stages = nn.ModuleList()
        dp_rates=[x.item() for x in torch.linspace(0, drop_path_rate, sum(depths))] 
        cur = 0
        for i in range(4):
            stage = nn.Sequential(
                *[Block(dim=dims[i], drop_path=dp_rates[cur + j], 
                layer_scale_init_value=layer_scale_init_value) for j in range(depths[i])]
            )
            self.stages.append(stage)
            cur += depths[i]

        self.norm = nn.LayerNorm(dims[-1], eps=1e-6)
        self.head = nn.Linear(dims[-1], num_classes)

        self.apply(self._init_weights)
        self.head.weight.data.mul_(head_init_scale)
        self.head.bias.data.mul_(head_init_scale)

    def _init_weights(self, m):
        if isinstance(m, (nn.Conv2d, nn.Linear)):
            nn.init.trunc_normal_(m.weight, std=.02)
            nn.init.constant_(m.bias, 0)

    def forward_features(self, x):
        features = []
        for i in range(4):
            x = self.downsample_layers[i](x)
            x = self.stages[i](x)
            features.append(x)
        return features

    def forward(self, x):
        features = self.forward_features(x)
        x = self.norm(features[-1].mean([-2, -1]))  # global average pooling
        x = self.head(x)
        return x

def convnext_tiny(pretrained=False, weights_path=None, **kwargs):
    """ConvNext Tiny model"""
    model = ConvNeXt(depths=[3, 3, 9, 3], dims=[96, 192, 384, 768], **kwargs)
    
    if pretrained and weights_path:
        print(f"🔄 Loading pretrained weights from {weights_path}")
        checkpoint = torch.load(weights_path, map_location="cpu")
        
        # Handle different checkpoint formats
        if "model" in checkpoint:
            state_dict = checkpoint["model"]
        else:
            state_dict = checkpoint
            
        model.load_state_dict(state_dict)
        print("✅ Pretrained weights loaded successfully!")
    elif pretrained:
        print("⚠️ Pretrained=True but no weights_path provided")
        
    return model


In [21]:
class ConvNextBackbone(nn.Module):
    """ConvNext backbone optimized for YOLO integration"""
    
    def __init__(self, weights_path=None, pretrained=True):
        super().__init__()
        
        # Create ConvNext Tiny with official architecture
        self.convnext = convnext_tiny(pretrained=pretrained, weights_path=weights_path)
        
        # Remove classification head (we only need features)
        self.convnext.norm = nn.Identity()
        self.convnext.head = nn.Identity()
        
        # ConvNext Tiny feature dimensions: [96, 192, 384, 768]
        self.feature_dims = [192, 384, 768]  # Last 3 stages for multi-scale detection
        target_dims = [256, 512, 1024]       # YOLO expected channels
        
        # Feature adaptation layers
        self.adapters = nn.ModuleList([
            nn.Sequential(
                nn.Conv2d(in_dim, out_dim, 1, bias=False),
                nn.BatchNorm2d(out_dim),
                nn.ReLU(inplace=True)
            ) for in_dim, out_dim in zip(self.feature_dims, target_dims)
        ])
        
        print("✅ ConvNext Tiny backbone ready for YOLO integration")
        print(f"📊 Feature channels: {self.feature_dims} → {target_dims}")

    def forward(self, x):
        # Extract multi-scale features from ConvNext
        features = self.convnext.forward_features(x)
        
        # Take last 3 feature maps: [Stage1: 192ch, Stage2: 384ch, Stage3: 768ch]
        selected_features = features[-3:]
        
        # Adapt to YOLO expected channels
        adapted_features = []
        for feat, adapter in zip(selected_features, self.adapters):
            adapted_feat = adapter(feat)
            adapted_features.append(adapted_feat)
        
        return adapted_features


In [22]:

def download_convnext_weights(model_name="convnext_tiny_1k"):
    """Download ConvNext weights if not present"""
    model_urls = {
        "convnext_tiny_1k": "https://dl.fbaipublicfiles.com/convnext/convnext_tiny_1k_224_ema.pth",
        "convnext_tiny_22k": "https://dl.fbaipublicfiles.com/convnext/convnext_tiny_22k_224.pth",
    }
    
    weights_file = f"{model_name}.pth"
    
    if os.path.exists(weights_file):
        print(f"✅ Found existing weights: {weights_file}")
        return weights_file
    
    if model_name in model_urls:
        print(f"📥 Downloading {model_name} weights...")
        checkpoint = torch.hub.load_state_dict_from_url(
            url=model_urls[model_name], 
            map_location="cpu", 
            file_name=weights_file
        )
        # Save to current directory for future use
        torch.save(checkpoint, weights_file)
        print(f"✅ Downloaded and saved: {weights_file}")
        return weights_file
    else:
        print(f"❌ Unknown model: {model_name}")
        return None


In [23]:
import torch
import torch.nn as nn
from ultralytics import YOLO


class WorkingConvNextYOLO(nn.Module):
    """Working YOLO model with ConvNeXt backbone - fixed iteration issues"""
    
    def __init__(self, yolo_path="yolov8n.pt", num_classes=2, convnext_weights_path=None, freeze_backbone=True, hyp=None):
        super().__init__()
        self.hyp = hyp
        self.args = SimpleNamespace(hyp=self.hyp)
        print(f"Creating Working ConvNext-YOLO...")
        
        # Load YOLO and get the underlying model
        yolo_wrapper = YOLO("yolov8n.pt")
        self.yolo = yolo_wrapper.model  # This is the DetectionModel
        self.head = self.yolo.model[-1]
        self.stride = self.head.stride 
        print(f"Original YOLO type: {type(self.yolo)}")
        
        # Access the model's sequential layers properly
        if hasattr(self.yolo, 'model'):
            model_layers = self.yolo.model  # This should be the sequential part
            print(f"Model layers type: {type(model_layers)}")
            print(f"Number of layers: {len(model_layers) if hasattr(model_layers, '__len__') else 'Unknown'}")
        
        # Replace backbone
        print("Replacing backbone with ConvNeXt...")
        #original_backbone = self.yolo.backbone
        self.yolo.backbone = ConvNextBackbone(
            weights_path=convnext_weights_path,
            pretrained=True if convnext_weights_path else False
        )
        print("swapped")
        
        # Freeze backbone if requested
        if freeze_backbone:
            print("Freezing ConvNeXt backbone...")
            for param in self.yolo.backbone.parameters():
                param.requires_grad = False
        
        # Update detection head
        self._update_detection_head(num_classes)
        
        # Ensure neck and head are trainable
        self._ensure_trainable_components()
        
        self._print_summary()
        
        self.model = nn.Sequential(
            *list(self.yolo.model[:-1]),  # backbone
            self.yolo.model[-1]           # detection head
        )
        self.model.args=self.args 
    
    def _update_detection_head(self, num_classes):
        """Update detection head for new classes"""
        print(f"Updating detection head: -> {num_classes} classes")
        
        # Find the Detect layer - it's usually the last layer in the model
        detect_layer = None
        if hasattr(self.yolo, 'model'):
            # Try to get the last layer
            if hasattr(self.yolo.model, '__getitem__'):
                detect_layer = self.yolo.model[-1]
            else:
                # Fallback: search through all modules
                for module in self.yolo.modules():
                    if hasattr(module, 'nc') and hasattr(module, 'cv2'):
                        detect_layer = module
                        break
        
        if detect_layer is None:
            print("Warning: Could not find detection layer!")
            return
        
        print(f"Found detection layer: {type(detect_layer)}")
        
        # Update detection layer properties
        old_nc = getattr(detect_layer, 'nc', 80)
        detect_layer.nc = num_classes
        detect_layer.no = num_classes + detect_layer.reg_max * 4
        
        # Update classification heads (cv2)
        if hasattr(detect_layer, 'cv2'):
            for i, cv2_layer in enumerate(detect_layer.cv2):
                if hasattr(cv2_layer, '__getitem__') and len(cv2_layer) > 0:
                    # Get the last conv layer in the cv2 sequence
                    last_conv = cv2_layer[-1]
                    if isinstance(last_conv, nn.Conv2d):
                        in_channels = last_conv.in_channels
                        # Replace with new conv layer for correct number of classes
                        cv2_layer[-1] = nn.Conv2d(in_channels, num_classes, 1, bias=True)
                        print(f"  Updated cv2[{i}]: {in_channels} -> {num_classes}")
        
        # Update regression heads (cv3) - these stay the same
        if hasattr(detect_layer, 'cv3'):
            for i, cv3_layer in enumerate(detect_layer.cv3):
                if hasattr(cv3_layer, '__getitem__') and len(cv3_layer) > 0:
                    last_conv = cv3_layer[-1]
                    if isinstance(last_conv, nn.Conv2d):
                        in_channels = last_conv.in_channels
                        cv3_layer[-1] = nn.Conv2d(in_channels, 4 * detect_layer.reg_max, 1, bias=True)
                        print(f"  Updated cv3[{i}]: {in_channels} -> {4 * detect_layer.reg_max}")
    
    def _ensure_trainable_components(self):
        """Ensure neck and head components are trainable"""
        print("Ensuring neck and head are trainable...")
        
        # Method 1: Unfreeze all non-backbone parameters
        for name, module in self.yolo.named_children():
            if name != 'backbone':
                for param in module.parameters():
                    param.requires_grad = True
        
        # Method 2: Specifically unfreeze detection layer
        for module in self.yolo.modules():
            if hasattr(module, 'nc') and hasattr(module, 'cv2'):  # This is Detect layer
                for param in module.parameters():
                    param.requires_grad = True
                break
        
        # Method 3: If model has sequential structure, unfreeze specific indices
        if hasattr(self.yolo, 'model') and hasattr(self.yolo.model, '__getitem__'):
            try:
                # Typical YOLOv8 structure: backbone is index 0, neck/head are later indices
                model_layers = self.yolo.model
                for i in range(1, len(model_layers)):  # Skip index 0 (backbone)
                    for param in model_layers[i].parameters():
                        param.requires_grad = True
            except Exception as e:
                print(f"Could not iterate through model layers: {e}")
    
    def _print_summary(self):
        """Print parameter summary"""
        total = sum(p.numel() for p in self.parameters())
        trainable = sum(p.numel() for p in self.parameters() if p.requires_grad)
        frozen = total - trainable
        
        print(f"\nParameter Summary:")
        print(f"  Total: {total:,} ({total/1e6:.1f}M)")
        print(f"  Trainable: {trainable:,} ({trainable/1e6:.1f}M)")
        print(f"  Frozen: {frozen:,} ({frozen/1e6:.1f}M)")
        print(f"  Trainable Ratio: {trainable/total*100:.1f}%")
        
        # Component breakdown
        print(f"\nComponent Breakdown:")
        if hasattr(self.yolo, 'backbone'):
            bb_total = sum(p.numel() for p in self.yolo.backbone.parameters())
            bb_trainable = sum(p.numel() for p in self.yolo.backbone.parameters() if p.requires_grad)
            print(f"  Backbone: {bb_total:,} total, {bb_trainable:,} trainable")
        
        # Count detection layer parameters
        for module in self.yolo.modules():
            if hasattr(module, 'nc') and hasattr(module, 'cv2'):
                det_total = sum(p.numel() for p in module.parameters())
                det_trainable = sum(p.numel() for p in module.parameters() if p.requires_grad)
                print(f"  Detection Head: {det_total:,} total, {det_trainable:,} trainable")
                break
                
    def init_criterion(self):
        """Initialize the loss function."""
        from ultralytics.utils.loss import v8DetectionLoss
        self.criterion = v8DetectionLoss(self)  # reads hyp from self.args.hyp
        return self.criterion
        
    def forward(self, x):
        return self.yolo(x)

def test_working_model():
    """Test the working model"""
    print("Testing Working ConvNext-YOLO...")
    
    try:
        # Download weights
        weights_path = download_convnext_weights("convnext_tiny_1k")
        
        # Create model
        model = WorkingConvNextYOLO(
            yolo_path="yolov8n.pt",
            num_classes=2,
            convnext_weights_path=weights_path,
            freeze_backbone=True
        )
        
        # Test forward pass
        print("\nTesting forward pass...")
        test_input = torch.randn(1, 3, 640, 640)
        
        if torch.cuda.is_available():
            model = model.cuda()
            test_input = test_input.cuda()
        
        model.eval()
        with torch.no_grad():
            output = model(test_input)
        
        print("Forward pass successful!")
        
        # Analyze output
        if isinstance(output, (list, tuple)):
            print(f"Output: {len(output)} elements")
            for i, out in enumerate(output):
                if hasattr(out, 'shape'):
                    print(f"  [{i}]: {out.shape}")
                else:
                    print(f"  [{i}]: {type(out)}")
        else:
            print(f"Output shape: {output.shape}")
        
        return model
        
    except Exception as e:
        print(f"Error: {e}")
        import traceback
        traceback.print_exc()
        return None

def debug_yolo_structure(yolo_path="yolov8n.pt"):
    """Debug YOLO model structure"""
    print("Debugging YOLO structure...")
    
    yolo = YOLO(yolo_path)
    model = yolo.model
    
    print(f"YOLO wrapper type: {type(yolo)}")
    print(f"Model type: {type(model)}")
    print(f"Model attributes: {dir(model)}")
    
    if hasattr(model, 'model'):
        inner_model = model.model
        print(f"Inner model type: {type(inner_model)}")
        if hasattr(inner_model, '__len__'):
            print(f"Inner model length: {len(inner_model)}")
            for i, layer in enumerate(inner_model):
                print(f"  Layer {i}: {type(layer).__name__}")
                if i > 10:  # Don't print too many
                    print(f"  ... and {len(inner_model) - i - 1} more layers")
                    break
    
    if hasattr(model, 'backbone'):
        print(f"Backbone type: {type(model.backbone)}")
    
    # Look for detection layer
    for i, module in enumerate(model.modules()):
        if hasattr(module, 'nc') and hasattr(module, 'cv2'):
            print(f"Found Detect layer at module {i}: nc={module.nc}")
            break




if __name__ == "__main__":
    # Debug first
    debug_yolo_structure()
    
    print("\n" + "="*60)
    
    # Test working model
    model = test_working_model()
    
    if model:
        # Run parameter analysis
        print("yes")

Debugging YOLO structure...
YOLO wrapper type: <class 'ultralytics.models.yolo.model.YOLO'>
Model type: <class 'ultralytics.nn.tasks.DetectionModel'>
Model attributes: ['T_destination', '__annotations__', '__call__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattr__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_apply', '_backward_hooks', '_backward_pre_hooks', '_buffers', '_call_impl', '_clip_augmented', '_compiled_call_impl', '_descale_pred', '_forward_hooks', '_forward_hooks_always_called', '_forward_hooks_with_kwargs', '_forward_pre_hooks', '_forward_pre_hooks_with_kwargs', '_get_backward_hooks', '_get_backward_pre_hooks', '_get_name', '_is_full_backward_hook', '_load_from_state_dict', '_loa

In [24]:


yaml_content = """
    path: /kaggle/input/home-fire-dataset
    train: train/images
    val: val/images
    test: test/images
    
    nc: 2
    names: ['fire', 'smoke']
    """

with open("/kaggle/working/home-fire.yaml", "w") as f:
    f.write(yaml_content)

                                                                            

In [25]:


class ConvNextYOLOTrainer(DetectionTrainer):
    def get_model(self, cfg=None, weights=None, verbose=True):
        """
        Override YOLO model loading with our custom ConvNeXt backbone.
        """
        # Initialize your custom YOLO model
        weights_path = download_convnext_weights("convnext_tiny_1k")
        model = WorkingConvNextYOLO(
            yolo_path="yolov8n.pt",
            num_classes=self.data["nc"],     # number of dataset classes
            convnext_weights_path = weights_path,      # or your convnext pretrained weights
            freeze_backbone=True             # freeze backbone if needed
        ).yolo

        
        keys = ["box", "cls", "dfl", "pose", "kobj"]
        hyp_dict = {k: getattr(self.args, k) for k in keys if hasattr(self.args, k)}
        print("⚠️ Building hyp from args:", hyp_dict)
        self.hyp = SimpleNamespace(**hyp_dict)
            
        print("done#########################")
        # Load weights if provided
        
        print("done2#########################################")
         # Build SimpleNamespace for hyp
        keys = ["box", "cls", "dfl", "pose", "kobj"]
        hyp_dict = {k: getattr(self.args, k) for k in keys if hasattr(self.args, k)}
        hyp_ns = SimpleNamespace(**hyp_dict)

        # Attach hyp to the model (NOT to args)
        model.model.args = hyp_ns
        print(model.args)
        # Initialize loss so the trainer can call it
        model.init_criterion()  # ensures self.criterion exists
        
        return model
import torch, gc
gc.collect()
torch.cuda.empty_cache()
args = dict(
    model="yolov8n.pt",   # dummy, will be replaced by get_model()
    data="/kaggle/working/home-fire.yaml",  # your dataset yaml
    epochs=50,
    imgsz=640,
    batch=8
)

trainer = ConvNextYOLOTrainer(overrides=args)
print("trainer is ready")

# Build hyp from args manually
# Do NOT assign to trainer.args.hyp
# trainer.args.hyp = trainer.hyp  <-- remove this line


trainer.train()

Ultralytics 8.3.198 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/kaggle/working/home-fire.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots

KeyboardInterrupt: 

In [26]:
from ultralytics import YOLO

ultra_model = YOLO("/kaggle/working/runs/detect/train2/weights/best.pt")

# Save only state_dict for future use
torch.save(ultra_model.model.state_dict(), "best_state_dict.pth")
state_dict = torch.load("best_state_dict.pth", map_location="cpu")
missing, unexpected = model.load_state_dict(state_dict, strict=False)


FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/working/runs/detect/train2/weights/best.pt'

In [ ]:

test_images = glob.glob("/kaggle/input/home-fire-dataset/test/images/*.jpg")  # adjust path if needed
print(len(test_images))
# Pick a few random samples
sample_images = random.sample(test_images, 5)

for img_path in sample_images:
    results = ultra_model(img_path)  # run inference
    
    # Save or display results
    results[0].show()   # display in notebook (OpenCV window in local env)
    results[0].save(filename=f"/kaggle/working/preds_{img_path.split('/')[-1]}")
for img_path in sample_images:
    results = model(img_path)
    annotated = results[0].plot()  # numpy array (BGR)
    
    # Convert BGR → RGB for matplotlib
    plt.imshow(cv2.cvtColor(annotated, cv2.COLOR_BGR2RGB))
    plt.axis("off")
    plt.title(img_path.split("/")[-1])
    plt.show()